In [1]:
%load_ext autoreload
%autoreload 2
import adanet

In [2]:
OUTPUT_DIR = './tmp/adanet'
datadir = './data'
is_tensorboard = True

In [3]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill

# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill

kill: (11643): Aucun processus de ce type
kill: (11649): Aucun processus de ce type


In [4]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, feature_shape = p8_util.load_dataset(filename_dataset)
print(x_train.shape, x_test.shape, feature_shape)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3)
(414, 224, 224, 3) (47, 224, 224, 3) (224, 224, 3)


#### Check a NN object candidate sample that will be created from Adanet algorithm.

In [5]:
import p8_util_config
p8_util_config.dict_adanet_config

{'adanet_feature_columns': None,
 'adanet_feature_shape': None,
 'adanet_tf_head': None,
 'adanet_lambda': 1e-05,
 'adanet_is_learn_mixture_weights': True,
 'adanet_initial_num_layers': 0,
 'adanet_num_layers': None,
 'adanet_nn_candidate': 2,
 'adanet_output_dir': './tmp/adanet',
 'adanet_nn_layer_config': {'nn_type': 'DNN',
  'nn_dropout_rate': 0.0,
  'nn_batch_norm': True,
  'nn_logit_dimension': 3,
  'nn_optimizer': <tensorflow.python.training.rmsprop.RMSPropOptimizer at 0x7ff9eb95f6a0>,
  'nn_seed': 42,
  'nn_initializer_name': 'xavier',
  'nn_layer_config': {'dnn_layer_num': 0, 'dnn_hidden_units': 128}}}

In [6]:
import p8_util

oNNAdaNetBuilder = p8_util.create_nn_builder(feature_shape,OUTPUT_DIR, layer_num=2)
oNNAdaNetBuilder.show()



Adanet outputdir     : ............................ ./tmp/adanet/DNN
Adanet output log    : ............................ ./tmp/adanet/DNN/DNN
Adanet iterations    : ............................ 2
NN type              : ............................ DNN
Features shape       : ............................ (224, 224, 3)
Number of layers     : ............................ 2
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Adanet regularization: ............................ 1e-05
Weights initializer  : ............................ xavier
Batch normalization  : ............................ True
Learn mixture weights: ............................ True


Number of layers     : ............................ 2
Hidden units         : ............................ 128


In [7]:
import p8_util_config
p8_util_config.ADANET_MAX_ITERATION_STEPS

2

In [8]:
import p8_util
import p8_util_config
import NNGenerator

#---------------------------------------------------------------------------------
# Update Adanet configuration with feature columns.
#---------------------------------------------------------------------------------
dict_nn_layer_config = p8_util_config.dict_adanet_config['adanet_nn_layer_config']
nb_class= dict_nn_layer_config['nn_logit_dimension']
nn_type = dict_nn_layer_config['nn_type']

feature_columns, loss_reduction, tf_head \
        = p8_util.get_tf_head(feature_shape, nb_class, nn_type=nn_type, feature_shape=feature_shape)

p8_util_config.dict_adanet_config['adanet_feature_columns'] = feature_columns
p8_util_config.dict_adanet_config['adanet_tf_head'] = tf_head
p8_util_config.dict_adanet_config['adanet_feature_shape'] = feature_shape

#---------------------------------------------------------------------------------

print("\n*** Number of classes= {} / NN type= {}".format(nb_class, nn_type))

adanet_estimator_config, output_dir_log = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)


*** Number of classes= 3 / NN type= DNN


In [9]:
import p8_util_config
print("Global steps= {}".format(p8_util_config.TRAIN_STEPS))
print("MAX Adanet steps= {}".format(p8_util_config.ADANET_MAX_ITERATION_STEPS))
print("")
p8_util_config.dict_adanet_config


Global steps= 5
MAX Adanet steps= 2



{'adanet_feature_columns': [NumericColumn(key='images', shape=(224, 224, 3), default_value=None, dtype=tf.float32, normalizer_fn=None)],
 'adanet_feature_shape': (224, 224, 3),
 'adanet_tf_head': <tensorflow_estimator.python.estimator.canned.head._MultiClassHeadWithSoftmaxCrossEntropyLoss at 0x7ff9eb827e80>,
 'adanet_lambda': 1e-05,
 'adanet_is_learn_mixture_weights': True,
 'adanet_initial_num_layers': 0,
 'adanet_num_layers': None,
 'adanet_nn_candidate': 2,
 'adanet_output_dir': './tmp/adanet',
 'adanet_nn_layer_config': {'nn_type': 'DNN',
  'nn_dropout_rate': 0.0,
  'nn_batch_norm': True,
  'nn_logit_dimension': 3,
  'nn_optimizer': <tensorflow.python.training.rmsprop.RMSPropOptimizer at 0x7ff9eb95f6a0>,
  'nn_seed': 42,
  'nn_initializer_name': 'xavier',
  'nn_layer_config': {'dnn_layer_num': 2, 'dnn_hidden_units': 128}}}

In [10]:
from adanet.ensemble import MixtureWeightType
from adanet.ensemble.weighted import ComplexityRegularizedEnsembler
        
        
if 'RNN' == nn_type :
    mixture_weight_type=MixtureWeightType.VECTOR
else :
    mixture_weight_type=MixtureWeightType.MATRIX

ensembler = ComplexityRegularizedEnsembler(mixture_weight_type=mixture_weight_type\
                                        , adanet_lambda=p8_util_config.dict_adanet_config['adanet_lambda'])

input_fn_param={'num_epochs':p8_util_config.NUM_EPOCHS,\
                'batch_size':p8_util_config.BATCH_SIZE,\
                'feature_shape': feature_shape,\
               }

train_input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param)

adanet_estimator = adanet.Estimator(
    head=p8_util_config.dict_adanet_config['adanet_tf_head'],
    subnetwork_generator=NNGenerator.NNGenerator(p8_util_config.dict_adanet_config),
    ensemblers        = [ensembler],
    
    max_iteration_steps=p8_util_config.ADANET_MAX_ITERATION_STEPS,
    
    evaluator=adanet.Evaluator(
        input_fn=train_input_fn,
        steps=None),
    config=  adanet_estimator_config)


*** NNGenerator() : feature_columns= [NumericColumn(key='images', shape=(224, 224, 3), default_value=None, dtype=tf.float32, normalizer_fn=None)]
*** INFO : MyGenerator : instantiation DONE!
INFO:tensorflow:Using config: {'_model_dir': './tmp/adanet/DNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff9eb8269b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_repli

I0514 08:15:46.609487 140713222428480 estimator.py:201] Using config: {'_model_dir': './tmp/adanet/DNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff9eb8269b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
import time
import tensorflow as tf
#---------------------------------------------------------------------
# Input function parameters stay same as for train_input_fn
#---------------------------------------------------------------------
test_input_fn=p8_util.input_fn("test", x_test, y_test,input_fn_param)
train_spec=tf.estimator.TrainSpec(
        input_fn= train_input_fn,
        max_steps=p8_util_config.TRAIN_STEPS)

eval_spec=tf.estimator.EvalSpec(
        input_fn= test_input_fn,
        steps=None,
        start_delay_secs=1,
        throttle_secs=1)

In [12]:
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(adanet_estimator, train_spec, eval_spec)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.


I0514 08:15:46.741724 140713222428480 estimator_training.py:185] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


I0514 08:15:46.745641 140713222428480 training.py:610] Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.


...

I0514 08:16:43.449563 140713222428480 estimator.py:1979] Saving dict for global step 5: accuracy = 0.31914893, average_loss = 1.098506, global_step = 5, loss = 1.0988889


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5: ./tmp/adanet/DNN/model.ckpt-5


I0514 08:16:43.453228 140713222428480 estimator.py:2039] Saving 'checkpoint_path' summary for global step 5: ./tmp/adanet/DNN/model.ckpt-5


INFO:tensorflow:Loss for final step: 1.0977621.


I0514 08:16:43.526874 140713222428480 estimator.py:359] Loss for final step: 1.0977621.


In [13]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("\n{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 56.79903841018677

ACCURACY: 0.3191489279270172

ARCHITECTURE/ADANET/ENSEMBLES: b'\nD\n\x13architecture/adanetB#\x08\x07\x12\x00B\x1d| DNN_layer_1 | DNN_layer_1 |J\x08\n\x06\n\x04text'

AVERAGE_LOSS: 1.098505973815918

LOSS: 1.098888874053955

GLOBAL_STEP: 5


In [14]:
#import tensorflow as tf
#help(tf.estimator.Estimator)

In [15]:
def ensemble_architecture(result):
  """Extracts the ensemble architecture from evaluation results."""

  architecture = result["architecture/adanet/ensembles"]
  # The architecture is a serialized Summary proto for TensorBoard.
  summary_proto = tf.summary.Summary.FromString(architecture)
  return summary_proto.value[0].tensor.string_val[0]

In [16]:
print("Architecture: {} / Accuracy= {} /Loss= {}".format(ensemble_architecture(results)\
                                                  , results['accuracy'], results['loss']))

Architecture: b'| DNN_layer_1 | DNN_layer_1 |' / Accuracy= 0.3191489279270172 /Loss= 1.098888874053955


In [17]:
# Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' : accuracy = 40% / Loss = 1.0883579
# Architecture: b'| CNNBase_layer_1 |' / Accuracy= 0.40425533056259155 /Loss= 1.081491470336914
# Architecture: b'| CNNBase_linear | CNNBase_layer_1 |' / Accuracy= 0.42553192377090454 /Loss= 1.1250361204147339

In [18]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(output_dir_log)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')